In [1]:
!sudo pip install transformers --upgrade

     |████████████████████████████████| 7.9MB 7.7MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 68.3MB/s eta 0:00:01
  Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [7]:
!sudo pip install safetensors --upgrade

     |████████████████████████████████| 1.4MB 6.8MB/s eta 0:00:01
  Found existing installation: safetensors 0.3.2
    Uninstalling safetensors-0.3.2:
      Successfully uninstalled safetensors-0.3.2


In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [2]:
import torch
from transformers import pipeline

# load model
model = AutoModelForCausalLM.from_pretrained(
    '/data/mistral/query-to-mql/exp-8/oct-30/merged-model',
#     quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
pipe("Reason for sales drop  Quarter1")

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Reason for sales drop  Quarter1: Given the context of the problem, we can assume that'}]

In [4]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
class ScoreTemplateExample(ScoreBase):
#     """
#     This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
#     """    
     def __init__(self):
        super().__init__()
        
        self.model_loaded = None
        
        import torch
        from transformers import pipeline
        
        # load model
        model = AutoModelForCausalLM.from_pretrained(
            '/data/mistral/query-to-mql/exp-8/oct-30/merged-model',
        #     quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        model.config.use_cache = False
        model.config.pretraining_tp = 1

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True,
                                                  # add_eos_token=True,
                                                  use_fast=False)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"

        # pipeline
        self.model_loaded = pipeline(task="text-generation", model=model, tokenizer=tokenizer,offload_folder="offload", device_map='auto')

 #############################################################       
 #       if self.model_loaded is None:
 #           print("LLM model loading from data section")
 #           self.model_loaded = pipeline("text-generation", model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f", torch_dtype=torch.bfloat16, device_map="auto")
#######################################################################
     def request_processing_fn(self, request) :
#         """
#         Processes Request Object -> List[Input data, ..]. It could be:
#                A List Mapping of All Value Can Be one of : 
#                    - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
#                    - List[numpy.array(), numpy.array(), ...]
#                    - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
#                    - List[ SingleSample, SingleSample]
                   
#         :return: (n_inputs, payload's)
        
#         Warnings:
#         1. Do not reshape your final output for single sample here, do it in prediction.
#            Else payloads will be invalidated for extraction at raw and extraction level.
#         """
         final_payload = []
         raw_payload = request.json["payload"]
         return (1, raw_payload) 
    
     def pre_processing_fn(self,payload):
#         # All preprocessing step must occur in this section
#         # Takes Single Sample -> Returns Single Sample
        
#         # Not Doing Any Preprocessing Hence Returned payload
#         print("payload is ", payload)
        
         return payload

     def prediction_fn(self,
                       model: Any,
                       input_query 
                       ):
#         """
#                 Does the main prediction on pre_processed_input(Single Sample) using supplied model .

#                 :param model: Supported Model
#                 :param pre_processed_input: Single Preprocessed Payload
#                 :return: Prediction Value From the model
                
#                 Important Notes:
#                 - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
#         """
#         model_loaded = self.model_loaded
#         mod = model_loaded
#         text = pre_processed_input #this is tuple we can iterate if there is number of input
        
    
##################################################################################################    
#          prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
#          text = input_query
#          prompt = prompt_template.format(query=text)
#         # We use a special <|end|> token with ID 49155 to denote ends of a turn
#          outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
#          preds = str(outputs[0]["generated_text"].split("<|assistant|>")[1])
        
        
#          print("prediction is \n: ",preds)
    
#          return preds
######################################################################################################

#     class Meta:    
         # List of Callables() can be attached For Calling After AnSd Before Scoring
#         def __init__(self):
#             self.name = "Pre Hooked Me !"
#             self.pre_call_hooks.append(self.print_)
#         def print_(self):
#             print(self.name)
#             pre_call_hooks = []
#             post_call_hooks = []
######################################################################################################
        # pipeline
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')
        output=pipe('Promopt')
        return output

In [6]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"Reason for sales drop  Quarter1"}

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_f

In [7]:
import time
t3 = time.time()
model_predictions = score_.score(None, req, dry_run=True)
t4 = time.time()
t4-t3

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Fields Marked Asterisk (*) Can Be Validated On Proper Input 



RuntimeError: Impossible to instantiate a pipeline with tokenizer specified but not the model as the provided tokenizer may not be compatible with the default model. Please provide a PreTrainedModel class or a path/identifier to a pretrained model when providing tokenizer.